In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout, Input# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
from scipy import stats
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%matplotlib inline
sns.set_style('darkgrid')
sns.set_palette('bone')

In [ ]:
f1=open('../input/train_V2.csv')
f2=open('../input/test_V2.csv')
df_train=pd.read_csv(f1)
df_test=pd.read_csv(f2)

In [ ]:
#删除空值
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# feature engineering

In [ ]:
# all_data = train.append(test, sort=False).reset_index(drop=True)
# del df_tatrain, test
# gc.collect()
all_data = df_train
all_data1 = df_test
all_data1['winPlacePerc'] = 0

In [ ]:
match = all_data.groupby('matchId')
all_data['killPlacePerc'] = match['kills'].rank(pct=True).values
all_data['walkDistancePerc'] = match['walkDistance'].rank(pct=True).values

In [ ]:
match1 = all_data1.groupby('matchId')
all_data1['killPlacePerc'] = match1['kills'].rank(pct=True).values
all_data1['walkDistancePerc'] = match1['walkDistance'].rank(pct=True).values

In [ ]:
all_data['_totalDistance'] = all_data['rideDistance'] + all_data['walkDistance'] + all_data['swimDistance']
all_data1['_totalDistance'] = all_data1['rideDistance'] + all_data1['walkDistance'] + all_data1['swimDistance']

In [ ]:
all_data['zombi'] = ((all_data['_totalDistance'] == 0) | (all_data['kills'] == 0)
                     | (all_data['weaponsAcquired'] == 0) 
                     | (all_data['matchType'].str.contains('solo'))).astype(int)
all_data['cheater'] = ((all_data['kills'] / all_data['_totalDistance'] >= 1)
                       | (all_data['kills'] > 30) | (all_data['roadKills'] > 10)).astype(int)
pd.concat([all_data['zombi'].value_counts(), all_data['cheater'].value_counts()], axis=1).T

In [ ]:
all_data1['zombi'] = ((all_data1['_totalDistance'] == 0) | (all_data1['kills'] == 0)
                     | (all_data1['weaponsAcquired'] == 0) 
                     | (all_data1['matchType'].str.contains('solo'))).astype(int)
all_data1['cheater'] = ((all_data1['kills'] / all_data1['_totalDistance'] >= 1)
                       | (all_data1['kills'] > 30) | (all_data1['roadKills'] > 10)).astype(int)
pd.concat([all_data1['zombi'].value_counts(), all_data1['cheater'].value_counts()], axis=1).T

In [ ]:
def fillInf(df, val):#删除inf值
    numcols = df.select_dtypes(include='number').columns
    cols = numcols[numcols != 'winPlacePerc']
    df[df == np.Inf] = np.NaN
    df[df == np.NINF] = np.NaN
    for c in cols: df[c].fillna(val, inplace=True)

In [ ]:
all_data['_healthAndBoosts'] = all_data['heals'] + all_data['boosts']
all_data['_killDamage'] = all_data['kills'] * 100 + all_data['damageDealt']
#all_data['_headshotKillRate'] = all_data['headshotKills'] / all_data['kills']
all_data['_killPlaceOverMaxPlace'] = all_data['killPlace'] / all_data['maxPlace']
all_data['_killsOverWalkDistance'] = all_data['kills'] / all_data['walkDistance']
#all_data['_killsOverDistance'] = all_data['kills'] / all_data['_totalDistance']
all_data['_walkDistancePerSec'] = all_data['walkDistance'] / all_data['matchDuration']

# suicide: solo and teamKills > 0
#all_data['_suicide'] = ((all_data['players'] == 1) & (all_data['teamKills'] > 0)).astype(int)

fillInf(all_data, 0)

In [ ]:
all_data1['_healthAndBoosts'] = all_data1['heals'] + all_data1['boosts']
all_data1['_killDamage'] = all_data1['kills'] * 100 + all_data1['damageDealt']
#all_data['_headshotKillRate'] = all_data['headshotKills'] / all_data['kills']
all_data1['_killPlaceOverMaxPlace'] = all_data1['killPlace'] / all_data1['maxPlace']
all_data1['_killsOverWalkDistance'] = all_data1['kills'] / all_data1['walkDistance']
#all_data['_killsOverDistance'] = all_data['kills'] / all_data['_totalDistance']
all_data1['_walkDistancePerSec'] = all_data1['walkDistance'] / all_data1['matchDuration']

# suicide: solo and teamKills > 0
#all_data['_suicide'] = ((all_data['players'] == 1) & (all_data['teamKills'] > 0)).astype(int)

fillInf(all_data1, 0)

In [ ]:
all_data.drop(['headshotKills','teamKills','roadKills','vehicleDestroys'], axis=1, inplace=True)
all_data.drop(['rideDistance','swimDistance','matchDuration'], axis=1, inplace=True)
all_data.drop(['rankPoints','killPoints','winPoints'], axis=1, inplace=True)

In [ ]:
all_data1.drop(['headshotKills','teamKills','roadKills','vehicleDestroys'], axis=1, inplace=True)
all_data1.drop(['rideDistance','swimDistance','matchDuration'], axis=1, inplace=True)
all_data1.drop(['rankPoints','killPoints','winPoints'], axis=1, inplace=True)

In [ ]:
all_data.drop(['Id','groupId','matchId'], axis=1, inplace=True)
all_data1.drop(['Id','groupId','matchId'], axis=1, inplace=True)

In [ ]:
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'
#mapper = lambda x: 'solo' if ('solo' in x) else 'team'
all_data['matchType'] = all_data['matchType'].map(mapper)
all_data1['matchType'] = all_data['matchType'].map(mapper)

In [ ]:
# 设置哑变量
a = pd.get_dummies(all_data['matchType'],prefix='matchType')
all_data = pd.concat([all_data,a],axis=1)
all_data1 = pd.concat([all_data1,a],axis=1)

In [ ]:
all_data.drop(['matchType'], axis=1, inplace=True)
all_data1.drop(['matchType'], axis=1, inplace=True)

In [ ]:
train_y = all_data['winPlacePerc']#吃鸡概率
all_data.drop(['winPlacePerc'], axis=1, inplace=True)
train_x = all_data

In [ ]:
predict = all_data1

In [ ]:
from sklearn import preprocessing
for item in list(train_x.columns):
#     all_data1[item].apply(preprocessing.scale,axis = 0)
#     all_data1[item] = preprocessing.scale(all_data1[item])
    train_x[item] = np.log1p(train_x[item])
    predict[item] = np.log1p(predict[item])

In [ ]:
predict.drop(['winPlacePerc'], axis=1, inplace=True)

In [ ]:
#删除空值
predict.dropna(inplace=True)
train_x.dropna(inplace=True)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, random_state=1)

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
print (linreg.intercept_)
print (linreg.coef_)

In [ ]:
y_pred = linreg.predict(X_test)
from sklearn import metrics
# 用scikit-learn计算MSE
print ("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print ("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#十折交叉验证
y = train_y#吃鸡概率
x = train_x
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(linreg, x, y, cv=10)
# 用scikit-learn计算MSE
print ("MSE:",metrics.mean_squared_error(y, predicted))
# 用scikit-learn计算RMSE
print ("RMSE:",np.sqrt(metrics.mean_squared_error(y, predicted)))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y, predicted)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
result = linreg.predict(predict)

In [ ]:
f3=open('../input/sample_submission_V2.csv')
submit=pd.read_csv(f3)

In [ ]:
sample_result = pd.DataFrame(result,columns = ['winPlacePerc'])

In [ ]:
submit['winPlacePerc'] = sample_result

In [ ]:
submit.to_csv(r'sample_submission_lineregression.csv', index=False)

# lasso 回归

In [ ]:
from sklearn.linear_model import Lasso,LassoCV,LassoLarsCV   # Lasso回归,LassoCV交叉验证实现alpha的选取，LassoLarsCV基于最小角回归交叉验证实现alpha的选取
# ========Lasso回归========
# model = Lasso(alpha=0.01)  # 调节alpha可以实现对拟合的程度
# model = LassoCV()  # LassoCV自动调节alpha可以实现选择最佳的alpha。
alpha = np.logspace(-3,2,10)
model = LassoCV(alphas=alpha,cv=5)  # LassoLarsCV自动调节alpha可以实现选择最佳的alpha
model.fit(X_train, y_train)   # 线性回归建模
print('系数矩阵:\n',model.coef_)
print('线性回归模型:\n',model)
print('最佳的alpha：',model.alpha_)  # 只有在使用LassoCV、LassoLarsCV时才有效


In [ ]:
# 使用模型预测
predicted_lasso = model.predict(predict)
# 存储文件
f4=open('../input/sample_submission_V2.csv')
submit_lasso=pd.read_csv(f4)
sample_result_lasso = pd.DataFrame(predicted_lasso,columns = ['winPlacePerc'])
submit_lasso['winPlacePerc'] = sample_result_lasso
submit_lasso.to_csv(r'sample_submission_lasso.csv', index=False)

# Ridge回归

In [ ]:
from sklearn.linear_model import RidgeCV,LassoCV#用这个自带交叉验证参数
from sklearn.model_selection import GridSearchCV#如果使用RidgeCV就不用GridSearchCV这个API了
#使用RidgeCV来建立参数
alpha = np.logspace(-3,2,10)#生成超参数，10的-3次方到10的2次方的等差数列
ridge = RidgeCV(alpha,cv=5)
ridge.fit(X_train,y_train)
ridge.alpha_#输出超参数的值
#使用Ridge配合GridSearchCV来做
from sklearn.linear_model import Ridge,Lasso
ridge_model = GridSearchCV(Ridge(),param_grid={'alpha':alpha},cv=5)
ridge_model.fit(X_train,y_train)
ridge_model.best_params_#验证模型效果

In [ ]:
# 使用模型预测
predicted_ridge = ridge_model.predict(predict)
# 存储文件
f4=open('../input/sample_submission_V2.csv')
submit_ridge=pd.read_csv(f4)
sample_result_ridge = pd.DataFrame(predicted_ridge,columns = ['winPlacePerc'])
submit_ridge['winPlacePerc'] = sample_result_ridge
submit_ridge.to_csv(r'sample_submission_ridge.csv', index=False)

# ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNetCV#用这个自带交叉验证参数
from sklearn.model_selection import GridSearchCV#如果使用RidgeCV就不用GridSearchCV这个API了
#ElasticNetCV
alpha = np.logspace(-3,2,10)#生成超参数，10的-3次方到10的2次方的等差数列
elasticNet = ElasticNetCV(alpha,cv=10)
elasticNet.fit(X_train,y_train)
elasticNet.alpha_#输出超参数的值
#使用Ridge配合GridSearchCV来做
from sklearn.linear_model import ElasticNet
elasticNet_model = GridSearchCV(ElasticNet(),param_grid={'alpha':alpha},cv=10)
elasticNet_model.fit(X_train,y_train)
elasticNet_model.best_params_#验证模型效果

In [ ]:
# 使用模型预测
predicted_elasticNet = elasticNet_model.predict(predict)
# 存储文件
f4=open('../input/sample_submission_V2.csv')
submit_elasticNet=pd.read_csv(f4)
sample_result_elasticNet = pd.DataFrame(predicted_elasticNet,columns = ['winPlacePerc'])
submit_elasticNet['winPlacePerc'] = sample_result_elasticNet
submit_elasticNet.to_csv(r'sample_submission_elasticNet.csv', index=False)

# xgboost

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, random_state=1)

In [ ]:
#定义一个函数帮助产生xgboost模型及其效果
def modelfit(alg, dtrain,result, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=result.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], result,eval_metric='auc')

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(result.values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(result, dtrain_predprob))

    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

#xgboost’s sklearn没有feature_importances，但是#get_fscore() 有相同的功能

In [ ]:
#通过固定的学习率0.1和cv选择合适的树的数量
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
#Choose all predictors except target & IDcols
predictors = [x for x in X_train.columns ]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X_train, y_train,predictors)
#作者调整后得到的树的值为140，如果这个值对于当前的系统而言太大了，可以调高学习率重新训练

In [ ]:
import xgboost as xgb
import sklearn
from sklearn.model_selection import GridSearchCV#如果使用RidgeCV就不用GridSearchCV这个API了
from sklearn.model_selection import train_test_split
X_dtrain, X_deval, y_dtrain, y_deval = train_test_split(train_x, train_y, random_state=1026, test_size=0.3)
cv_params = {'n_estimators': [400, 500, 600, 700, 800]}
# cv_params = {'n_estimators': [400]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 2019,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

model_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_dtrain, y_dtrain)
evalute_result = optimized_GBM.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

# lightGBM

In [ ]:
from sklearn.model_selection import cross_val_score
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_holdout, y_train, y_holdout =  train_test_split(train_x, train_y, random_state= 666,test_size=0.2)

In [ ]:
%%time
import lightgbm as lgb
lgtrain = lgb.Dataset(X_train, label=y_train.reset_index(drop=True))
res = lgb.cv({'metric': 'mae'},lgtrain, nfold=5,stratified=False,seed=666)
print("Mean score:",res['l1-mean'][-1])

In [ ]:
gridParams = {
    'num_leaves': [30,50,100], 'max_depth': [-1,8,15], 
    'min_data_in_leaf': [100,300,500], 'max_bin': [250,500], 
    'lambda_l1': [0.01], 'num_iterations': [5], 
    'nthread': [4], 'seed': [666],
    'learning_rate': [0.05], 'metric': ['mae'],
    "bagging_fraction" : [0.7], "bagging_seed" : [0], "colsample_bytree" : [0.7]
    }
model = lgb.LGBMRegressor()
grid = GridSearchCV(model, gridParams,
                    verbose=1,
                    cv=5)

In [ ]:
%%time
grid.fit(X_train.iloc[:500000,:], y_train.iloc[:500000])

In [ ]:
print("Best params:", grid.best_params_)
print("\nBest score:", grid.best_score_)
params = grid.best_params_

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
model = lgb.LGBMRegressor(learning_rate=0.05,nthread=4)

def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
    facecolor=lines[0].get_color(), alpha=0.2)
    
def plot_learning_curve():
    train_sizes = [1000,5000,10000,50000,100000,500000]
    N_train, val_train, val_test = learning_curve(model,
    X_train, y_train, train_sizes=train_sizes, cv=5,
    scoring='neg_mean_absolute_error')
    plot_with_err(N_train, abs(val_train), label='training scores')
    plot_with_err(N_train, abs(val_test), label='validation scores')
    plt.xlabel('Training Set Size'); plt.ylabel('MAE')
    plt.legend()

plot_learning_curve()

In [ ]:
def iter_vs_score(num_iterations):
    val_train, val_test = validation_curve(model, X_train[:500000], y_train[:500000],
        'num_iterations', num_iterations, cv=4,scoring='neg_mean_absolute_error', verbose=1)
    plot_with_err(num_iterations, abs(val_test), label='validation scores')
    plot_with_err(num_iterations, abs(val_train), label='training scores')
    plt.xlabel('Number of iterations'); plt.ylabel('MAE')
    plt.legend();
    plt.show();

num_iterations_small = [5,10,20,30,100,200]
iter_vs_score(num_iterations_small)
num_iterations_big = [500,1000,5000,10000]
iter_vs_score(num_iterations_big)

In [ ]:
num_iterations_big = [10000,12000,15000,20000]
iter_vs_score(num_iterations_big)

# NN

In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout, Input

In [ ]:
%%time
# create NN_model
NN_model = Sequential()
NN_model.add(Dense(x_train.shape[1],  input_dim = x_train.shape[1], activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))

# output Layer
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
%%time
NN_model.fit(x=x_train, y=y, batch_size=1000,
             epochs=30, verbose=1, callbacks=callbacks_list,
             validation_split=0.15, validation_data=None, shuffle=True,
             class_weight=None, sample_weight=None, initial_epoch=0,
             steps_per_epoch=None, validation_steps=None)
del x_train, y
gc.collect()

In [ ]:
x_test, _ = feature_engineering(False)
scaler.transform(x_test)
print("x_test", x_test.shape, x_test.max(), x_test.min())
np.clip(x_test, out=x_test, a_min=-1, a_max=1)
print("x_test", x_test.shape, x_test.max(), x_test.min())